In [98]:
#CONSTANTS TO RUN THE PAGE

PATH_TO_DATA="/Users/williamcochran/Code/learn_stuff/connect6/data/"
MOVES_TO_DETERMINE_GAME=10
FIRST_MOVE="J10"
BLACK=1
WHITE=-1
EMPTY=0

def swap(color):
    return color*-1



In [99]:
#Let's read some data!
import os

# File finder and reader.. returns a list of moves
def parse_game_file (filename):
    with open(filename, "r") as f:
        lines = f.readlines()
        games = []
        for line in lines:
            if line[0] != "(" and line[-1] != ")":
                print("Found a bad line: {}".format(line))
            trimmed = line[1:-2]
            keys=trimmed.split(';')
            keys = keys[3:]
            moves = [k[2:-1] for k in keys]
            if len(moves)>0:
                games.append(moves)
    return games

# Look for all files in the data path
def read_all_data (path=PATH_TO_DATA):
    games = []
    all_files = os.listdir(path)
    data_files = [file for file in all_files if file.endswith(".data")]
    for file in data_files:
        print ("Reading {}".format(file))
        games.extend(parse_game_file (PATH_TO_DATA + file))
    return games



In [100]:
# Board manipulation helper functions
def make_board():
    board = []
    for a in range(19):
        b = []
        for c in range(19):
            b.append(EMPTY)
        board.append(b)
    board[9][9] = BLACK
    return board


def get_stone(move_cluster,spot):
    x = ord(move_cluster[spot]) - ord('a')
    y = ord(move_cluster[spot+1]) - ord('1')
    ans = spot + 2
    if spot+2 < len(move_cluster):
        if move_cluster[spot+2].isdigit():
            ans = ans + 1
            # Be sure to add 1 to the digit since a single digit 
            # number is assumed to start at "1" instead of "0"
            y = (y+1) * 10 + ord(move_cluster[spot+2]) - ord('1')
    return ((x,y),ans)

def count_stones (board):
    return sum([sum(k) for k in board])

def return_moves(move_cluster):
    (stone1,offset) = get_stone(move_cluster,0)
    (stone2,offset) = get_stone(move_cluster,offset)
    return (stone1,stone2)

def pretty_print(board):
    for x in board:
        line = "  "
        for y in x:
            if y == -1:
                line = line + "O "
            elif y == 1:
                line = line + "X "
            else:
                line = line + "+ "
        print(line)

def apply_move(board, stone, value):
    (x,y) = stone
    board[x][y] = value


In [101]:

# Time to sort through all the games to find complete games... 
# For games with more than MOVES_TO_DETERMINE_GAME moves to remove trivial stuff.
import copy

def separate_wins(games):
    black_wins = []
    white_wins = []
    
    for game in games:
        if len(game)>=MOVES_TO_DETERMINE_GAME:
            if len(game) % 2 == 0:
                black_wins.append(game)
            else:
                white_wins.append(game)

    print ("Found {} games for black wins and {} games for white wins".format(len(black_wins),len(white_wins)))
    return (black_wins,white_wins)



def build_board_list(game):
    board_list = []
    board = make_board()
    value = WHITE
    for move_cluster in game:
        board = copy.deepcopy(board)
        (stone1,stone2) = return_moves(move_cluster)
        apply_move(board,stone1,value)
        apply_move(board,stone2,value)
        value = swap(value)
        board_list.append(board)
    return board_list

def make_boards_for_list(game_list):
    boards = []
    for game in game_list:
        boards.extend(build_board_list(game))
    return boards

def make_boards_for_wins(games):
    (black_wins,white_wins) = separate_wins(games)
    black_board_list = make_boards_for_list(black_wins)
    white_board_list = make_boards_for_list(white_wins)
    return (black_board_list,white_board_list)



In [ ]:
# time for the machine to do some learning!
import math
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

games = read_all_data()
print ("Loaded {} games.".format(len(games)))

(black_list,white_list) = make_boards_for_wins(games)
X = black_list
y = [1 for _ in range(len(black_list))]
X.extend(white_list)
y.extend([-1 for _ in range(len(white_list))])


# Convert lists to NumPy arrays and reshape X to include a channel dimension
X = np.array(X)  # Your list of 19x19 boards
y = np.array(y)  # Your classifications (1 or -1)

X = np.expand_dims(X, -1)  # Add a single channel dimension

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv2D(256, (3, 3), activation='relu', input_shape=(19, 19, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(1, activation='tanh')  # Output layer with tanh activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Using MSE as the loss function
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")


Reading 11936.data
Reading 166925.data
Reading 15609.data
Reading 35959.data
Reading 36197.data
Reading 142840.data
Reading 14287.data
Loaded 21478 games.
Found 9572 games for black wins and 9678 games for white wins
Epoch 1/100
10852/10852 [==============================] - 86s 8ms/step - loss: 1.4885 - accuracy: 0.1296 - val_loss: 1.8982 - val_accuracy: 0.0000e+00
Epoch 2/100
10852/10852 [==============================] - 84s 8ms/step - loss: 1.8935 - accuracy: 0.0000e+00 - val_loss: 1.8982 - val_accuracy: 0.0000e+00
Epoch 3/100
10852/10852 [==============================] - 84s 8ms/step - loss: 1.8935 - accuracy: 0.0000e+00 - val_loss: 1.8982 - val_accuracy: 0.0000e+00
Epoch 4/100
10852/10852 [==============================] - 83s 8ms/step - loss: 1.8935 - accuracy: 0.0000e+00 - val_loss: 1.8982 - val_accuracy: 0.0000e+00
Epoch 5/100
10852/10852 [==============================] - 83s 8ms/step - loss: 1.8935 - accuracy: 0.0000e+00 - val_loss: 1.8982 - val_accuracy: 0.0000e+00
Epoch 6